In [1]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
from sklearn.cross_validation import train_test_split
import re
import operator
style.use('ggplot')

# read in csv as dataframe
# c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')
c = pd.read_csv("../blocking/candidate_set_with_added_features.csv", encoding="ISO-8859-1", index_col='_id')
#c = c.dropna()
c.head()

c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,ltable_pop_trend,...,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,NaN,...,0.16,-10.0,3.0,0,0.5,1.0,0.0,1,0.0,1
75598,4363,9,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1
75600,4365,9,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\r\n The major threat is over-collectin...,['\n The Shapotou location is within a nati...,Decreasing,...,0.08,-16.0,1.0,0,0.5,1.0,0.0,1,0.0,1
78778,7543,9,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\r\n The major threat is habitat loss a...,['\n It is listed on Appendix III of the Be...,Decreasing,...,0.12,-14.0,2.0,0,0.5,1.0,0.0,0,0.0,1
83513,4363,10,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\r\n The major threat is drainage of we...,"[""\n It is listed on Appendix III of the Be...",Decreasing,...,0.16,-2.0,10.0,1,1.0,0.0,1.0,1,1.0,1


In [2]:
c.columns

Index(['ltable_id', 'rtable_id', 'ltable_name', 'ltable_genus',
       'ltable_family', 'ltable_ecology', 'ltable_countries',
       'ltable_threat_paragraph', 'ltable_conservation_paragraph',
       'ltable_pop_trend', 'ltable_status', 'ltable_country_count',
       'ltable_scientific_name', 'rtable_scientific_name', 'rtable_name',
       'rtable_kingdom', 'rtable_phylum', 'rtable_class', 'rtable_order',
       'rtable_family', 'rtable_genus', 'rtable_size', 'rtable_threats',
       'rtable_conservation', 'rtable_threat_keywords',
       'rtable_conservation_keywords', 'rtable_status', 'rtable_countries',
       'rtable_country_count', 'rtable_tCount', '_id.1', 'ltable_id.1',
       'rtable_id.1', 'name_name_jac_qgm_3_qgm_3',
       'name_name_jac_dlm_dc0_dlm_dc0', 'countries_countries_jac_qgm_3_qgm_3',
       'countries_countries_cos_dlm_dc0_dlm_dc0',
       'countries_countries_jac_dlm_dc0_dlm_dc0', 'countries_countries_mel',
       'countries_countries_lev_dist', 'countries_countri

In [3]:
def list_duplicates(seq):
  seen = set()
  seen_add = seen.add
  # adds all elements it doesn't know yet to seen and all other to seen_twice
  seen_twice = set( x for x in seq if x in seen or seen_add(x) )
  # turn the set into a list (as requested)
  return list( seen_twice )

list_duplicates(c.columns)

[]

In [4]:
matches = 0
nonmatches = 0;

gold = None
DS = None
labels = []


while matches < 60:
    print('matches:', matches)
    gold,unlabeled = train_test_split(c, train_size=0.1551, test_size=0.8449)
    #gold,unlabeled = train_test_split(c, train_size=0.486, test_size=0.514)

    #Automatically label the training data
    matches = 0;
    nonmatches = 0;
    labels = []

    
    for index, row in gold.iterrows():
        if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
            labels.append(1)
            matches = matches + 1
        else:
            labels.append(0)
            nonmatches = nonmatches + 1

#     matches = 50

gold['label'] = labels
#print('number of matches in training set:', matches)
#print('number of nonmatches in training set:', nonmatches)

matches: 0
matches: 30
matches: 45
matches: 43
matches: 37
matches: 37
matches: 34
matches: 38
matches: 41
matches: 44
matches: 41
matches: 42
matches: 40
matches: 46
matches: 50
matches: 47
matches: 36
matches: 49
matches: 46
matches: 39
matches: 39
matches: 46
matches: 50
matches: 38
matches: 30
matches: 39
matches: 34
matches: 44
matches: 46
matches: 40
matches: 48
matches: 41
matches: 40
matches: 39
matches: 40
matches: 46
matches: 32
matches: 39
matches: 47
matches: 35
matches: 39
matches: 39
matches: 37
matches: 43
matches: 32
matches: 34
matches: 46
matches: 33
matches: 36
matches: 38
matches: 42
matches: 39
matches: 39
matches: 49
matches: 40
matches: 37
matches: 36
matches: 35
matches: 30
matches: 30
matches: 51
matches: 28
matches: 44
matches: 43
matches: 28
matches: 40
matches: 35
matches: 49
matches: 42
matches: 39
matches: 41
matches: 46
matches: 34
matches: 45
matches: 45
matches: 41
matches: 33
matches: 46
matches: 32
matches: 41
matches: 26
matches: 30
matches: 37
match

c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
for col in c.columns:
    print(col)

ltable_id
rtable_id
ltable_name
ltable_genus
ltable_family
ltable_ecology
ltable_countries
ltable_threat_paragraph
ltable_conservation_paragraph
ltable_pop_trend
ltable_status
ltable_country_count
ltable_scientific_name
rtable_scientific_name
rtable_name
rtable_kingdom
rtable_phylum
rtable_class
rtable_order
rtable_family
rtable_genus
rtable_size
rtable_threats
rtable_conservation
rtable_threat_keywords
rtable_conservation_keywords
rtable_status
rtable_countries
rtable_country_count
rtable_tCount
_id.1
ltable_id.1
rtable_id.1
name_name_jac_qgm_3_qgm_3
name_name_jac_dlm_dc0_dlm_dc0
countries_countries_jac_qgm_3_qgm_3
countries_countries_cos_dlm_dc0_dlm_dc0
countries_countries_jac_dlm_dc0_dlm_dc0
countries_countries_mel
countries_countries_lev_dist
countries_countries_lev_sim
countries_countries_nmw
countries_countries_sw
country_count_country_count_exm
country_count_country_count_anm
country_count_country_count_lev_dist
country_count_country_count_lev_sim
status_match
country_overlap
co

In [6]:
# gold.to_csv('./gold.csv')
gold.to_csv('./gold.csv')
# unlabeled.to_csv('./unlabeled.csv')
unlabeled.to_csv('./unlabeled.csv')


In [7]:
# remove unnecessary features
# write out gold and unlabeled csvs with only selected features for matcher

#cols_to_keep = ['country_count_sim', 
#                'status_match', 
#                'country_overlap', 
#                'name_name_jac_dlm_dc0_dlm_dc0', 
#                'name_name_jac_qgm_3_qgm_3',
#                'countries_countries_jac_dlm_dc0_dlm_dc0', 
#                'countries_countries_jac_qgm_3_qgm_3'
#                ]

#cols_to_keep version 2
cols_to_keep = ['name_name_jac_qgm_3_qgm_3',
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'countries_countries_jac_qgm_3_qgm_3',
                'countries_countries_cos_dlm_dc0_dlm_dc0',
                'countries_countries_jac_dlm_dc0_dlm_dc0', 
                'countries_countries_mel',
                'countries_countries_lev_dist', 
                'countries_countries_lev_sim',
                'countries_countries_nmw', 
                'countries_countries_sw',
                'country_count_country_count_exm', 
                'country_count_country_count_anm',
                'country_count_country_count_lev_dist',
                'country_count_country_count_lev_sim', 
                'status_match',
                'country_overlap', 
                'country_count_sim']
# FOR NOW, just use these while we wait for other features
# cols_to_keep_gold = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3',
#                 'label']
gold_reduced = gold[cols_to_keep + ['label']] 

# cols_to_keep_unlabeled = ['country_count_sim', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3']
unlabeled_reduced = unlabeled[cols_to_keep]

In [8]:
# gold_reduced.to_csv('./gold_reduced.csv')
gold_reduced.to_csv('./gold_reduced.csv')
# unlabeled_reduced.to_csv('./unlabeled_reduced.csv')
unlabeled_reduced.to_csv('./unlabeled_reduced.csv')

In [9]:
#using features for matcher
train,test = train_test_split(gold, train_size=0.7, test_size=0.3)
len(train)

train_reduced = train[cols_to_keep + ['label']] 
test_reduced = test[cols_to_keep]

In [10]:
train.to_csv('./train.csv')
test.to_csv('./test.csv')
train_reduced.to_csv('./train_reduced.csv')
test_reduced.to_csv('./test_reduced.csv')

In [12]:
train_reduced


,name_name_jac_qgm_3_qgm_3,name_name_jac_dlm_dc0_dlm_dc0,countries_countries_jac_qgm_3_qgm_3,countries_countries_cos_dlm_dc0_dlm_dc0,countries_countries_jac_dlm_dc0_dlm_dc0,countries_countries_mel,countries_countries_lev_dist,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim,label
_id,,,,,,,,,,,,,,,,,,
7281363,NaN,NaN,0.076923,0.0,0.0,0.483487,26,0.161290,-19.0,5.0,0,0.500000,1.0,0.0,1,1.0,1,0
7278036,NaN,NaN,0.000000,0.0,0.0,0.431900,28,0.096774,-22.0,2.0,0,0.500000,1.0,0.0,0,0.0,1,0
2041033,NaN,NaN,0.000000,0.0,0.0,0.441667,15,0.062500,-5.0,2.0,1,1.000000,0.0,1.0,0,0.0,1,0
4034499,0.285714,0.333333,0.000000,0.0,0.0,0.433211,31,0.114286,-18.0,2.0,0,0.500000,1.0,0.0,1,0.0,1,0
2029105,NaN,NaN,0.000000,0.0,0.0,0.383572,39,0.093023,-26.0,1.0,1,1.000000,0.0,1.0,1,0.0,1,0
1369586,NaN,NaN,0.000000,0.0,0.0,0.513095,33,0.153846,0.0,4.0,0,0.333333,1.0,0.0,1,0.0,0,0
4992441,NaN,NaN,0.000000,0.0,0.0,0.443254,21,0.125000,-6.0,2.0,0,0.500000,1.0,0.0,0,0.0,1,0
5992303,NaN,NaN,0.000000,0.0,0.0,0.455556,11,0.083333,-1.0,1.0,1,1.000000,0.0,1.0,0,0.0,1,0
4414743,0.068966,0.250000,0.018519,0.0,0.0,0.549383,51,0.055556,-42.0,3.0,0,0.500000,1.0,0.0,0,0.0,1,0
